In [1]:
from torch.utils.data import DataLoader
from generator_class_v2 import Dataset,my_collate,AverageMeter

from torch_baidu_ctc import CTCLoss
#from torch.nn import CTCLoss

from ocr_model_v2 import cnn_attention_ocr
#from torch.nn import CTCLoss
import torch
import torch.nn  as nn
import torch.optim as optim
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau,ExponentialLR
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter

def show(img):
    npimg = img.numpy()
    plt.figure()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

from evaluation import wer_eval

In [2]:
###Evaluation methods 
cnn=cnn_attention_ocr(model_dim=64,nclasses=93,n_layers=8)
cnn=cnn.cuda().train()
count_parameters(cnn)

3543584

In [3]:
writer = SummaryWriter(log_dir="/home/leander/AI/repos/OCR-CNN/logs2/retry_no_aug")

In [4]:
ctc_loss = CTCLoss(reduction="mean",average_frames=True)
#Good initial is 5e5 
optimizer = optim.Adam(cnn.parameters(), lr=5e-5)

In [5]:
ave_total_loss = AverageMeter()
CER_total= AverageMeter()

n_iter=0
batch_size=4

In [6]:
#cnn.load_state_dict(torch.load("contiued_training_on_heavy_agment_CER4%_22_6kite.pt"))

In [ ]:
for epochs in range(10000):
    #Initing the dataset actually downloads a bunch of data and creats the images 
    print("getting data")
    ds=Dataset(batch_size,epoch_size=1000,random_strings=True,num_words=5)
    #Then we set up our own custom dataloader, with a custom collate, which packs the data
    #(does the padding) Should work with variable number of widths. 

    trainset = DataLoader(dataset=ds,
                          #Multi processing worker leads to an error with CTC Loss
                          #num_workers=6,
                          batch_size=batch_size,
                          shuffle=False,
                          collate_fn=my_collate)
    gen = iter(trainset)
    print("train start")
    for ge in gen:
        if ge[0].shape[3]<800:
            #break
            #DONT FORGET THE ZERO GRAD!!!!
            optimizer.zero_grad()
            #Get Predictions
            log_probs = cnn(ge[0]).permute((2,0,1))

            #Targets have to be CPU 
            targets = ge[1].cpu()

            #Get the Lengths/2 becase this is how much we downsample the width
            input_lengths = ge[2]/2
            target_lengths = ge[3]
            #Get the CTC Loss 
            loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
            loss.backward()

            optimizer.step()
            
            #Save Loss 
            ave_total_loss.update(loss.data.item())
            #print(ave_total_loss.average())
            #Write it to Tensorborad 
            writer.add_scalar("total_loss", ave_total_loss.average(), n_iter) 
            n_iter=n_iter+1
            
            
            #Here we Calculate the Character error rate
            cum_len=np.cumsum(target_lengths)
            targets=np.split(ge[1].cpu(),cum_len[:-1])
            wer_list=[]
            for j in range(log_probs.shape[1]):
                wer_list.append(wer_eval(log_probs[:,j,:][0:input_lengths[j],:],targets[j]))
            #print(np.average(wer_list))
            
            CER_total.update(np.average(wer_list))
            writer.add_scalar("CER", CER_total.average(), n_iter)
            #print(CER_total.average())
            del log_probs
            

getting data
train start


/home/leander/AI/repos/Pytorch-OCR-Fully-Conv/evaluation.py:41: RuntimeWarning: divide by zero encountered in true_divide
  return we/len(preds)


getting data
train start
getting data
train start


In [16]:
m=cnn.state_dict()

In [17]:
torch.save(m,"contiued_training_on_heavy_agment_CER2%_40kite.pt")